## Zuiveringsmodel

Je hebt toegang gekregen tot een model dat de stikstofverwijdering in een RWZI simuleert. Aan de hand van gegevens over het influent (volume en ammonium), de hoeveelheid beluchting en de recirculatie kan dit model de kwaliteit van het effluent berekenen.

In [ ]:
# imports en inladen data

import pandas as pd
import numpy as np

from model_zuivering import zuivering

path = r'../data/influent.csv'
influent_df = pd.read_csv(path)
influent_df['Datumtijd'] = pd.to_datetime(influent_df['Datumtijd'])

In [ ]:
def bereken_energieverbruik(beluchting_W, recirculatie_W, t=15*60):
    '''
    Berekent de hoeveelheid energie die wordt verbruikt gedurende een periode dat de pompen aanstaan

    Parameters
    ----------
        beluchting_W: int
            Hoeveelheid beluchting in W
        recirculatie_W: int
            Recirculatie in W
        t: int
            Tijd in s dat de pompen aanstaan

    Returns
    ----------
        energie_verbruik_in_kWh: float
            Energieverbruik in kWh
    '''

    energie_verbruik_in_J = t * (beluchting_W + recirculatie_W)  # J
    energie_verbruik_in_kWh =  0.0000002777777777777778 * energie_verbruik_in_J  # kWh
    return energie_verbruik_in_kWh


In [ ]:
def statische_optimalisatie(influent_df, vermogen_beluchting, vermogen_recirculatie):
    '''
    Rekent de stikstofverwijdering van de zuivering door aan de hand van influent data en statische waardes voor het vermogen van de pompen. 

    Parameters
    ----------
        influent_df: DataFrame
            Gegevens over het influent: 
                volume: float
                    Volume van het influent gedurende 15 minuten in m3
                NH4: float
                    Concentratie ammonium in het influent in mg/l
        vermogen_beluchting: int
            Hoeveelheid beluchting in W
        vermogen_recirculatie: int
            Recirculatie in W

    Returns
    ----------
        effluent_df: DataFrame
                volume: float
                    Volume van het influent gedurende 15 minuten in m3
                NH4: float
                    Concentratie ammonium in het influent in mg/l
                effluent_n_tot: float
                    Concentratie stikstof in het effluent in mg/l
                energieverbruik: float
                    Hoeveelheid verbruikte energie in kWh
                effluent_ok: bool
                    Of de concentratie stikstof in het effluent binnen de norm is
    '''
    
    rwzi = zuivering.Zuivering()
    effluent_n_tot = list()
    energieverbruik = list()

    for _, row in influent_df.iterrows(): 
        effluent_n_tot.append(rwzi.step(row['volume'], row['NH4'], vermogen_beluchting, vermogen_recirculatie))
        energieverbruik.append(bereken_energieverbruik(vermogen_beluchting, vermogen_recirculatie))  # kWh

    effluent_df = influent_df.copy()
    effluent_df['effluent_N_tot'] = effluent_n_tot
    effluent_df['energieverbruik'] = energieverbruik
    effluent_df['effluent_ok'] = np.where(effluent_df['effluent_N_tot'] <= 10, True, False)

    return effluent_df



## Statische optimalisatie

Vanaf hier kan je gaan experimenteren met de beluchting en recirculatie. 

In [ ]:

# probeer eens andere waardes voor deze parameters. Blijft het effluent schoon? 
vermogen_beluchting = 30000  # W
vermogen_recirculatie = 5000  # W

effluent_df = statische_optimalisatie(influent_df, vermogen_beluchting, vermogen_recirculatie)
totaal_energieverbruik = effluent_df['energieverbruik'].sum()
proportie_schoon_effluent = len(effluent_df[effluent_df['effluent_ok']]) / len(effluent_df)


print('Totaal energieverbruik: {:.0f}kWh'.format(totaal_energieverbruik))
print('Percentage van de tijd dat de effluentkwaliteit binnen de norm bleef: {:.1f}%'.format(100*proportie_schoon_effluent))

---

## Dynamische optimalisatie

Vanaf hier kun je experimenteren met een dynamische optimalisatie. Je mag aannemen dat je tot 6 uur in de toekomst het influent kan voorspellen.

In [ ]:
def dynamische_optimalisatie(influent):
    '''
    Berekent de hoeveelheid beluchting en recirculatie aan de hand van het recente en aankomende influent

    Parameters
    ----------
        influent: DataFrame
            Gegevens over het influent van de afgelopen en komende 6 uur: 
                volume: float
                    Volume van het influent gedurende 15 minuten in m3
                NH4: float
                    Concentratie ammonium in het influent in mg/l

    Returns
    ----------
        beluchting: int
            Hoeveelheid beluchting in W
        recirculatie: int
            Recirculatie in W
    '''
    #------------------------------------------------------------------------------
    #TODO vanaf hier kan je alles proberen wat je wil

    gemiddeld = influent.mean()
    volume = gemiddeld['volume']
    ammonium = gemiddeld['NH4']

    
    beluchting = 1000 * volume + 100 * ammonium
    recirculatie = 10 * volume +  100 * ammonium
    #------------------------------------------------------------------------------

    return min(beluchting, 40000), min(recirculatie, 5500)  # maximale capaciteit meegenomen

Voer onderstaande cel uit om je dynamische optimalisatie toe te passen en om te zetten naar een effluent df

In [ ]:

rwzi = zuivering.Zuivering()
effluent_n_tot = list()
energieverbruik = list()

for index, row in influent_df.iterrows(): 
    vermogen_beluchting, vermogen_recirculatie = dynamische_optimalisatie(influent_df.loc[index-6*4:index+6*4-1,:])  # geeft recente en toekomstig influent mee

    effluent_n_tot.append(rwzi.step(row['volume'], row['NH4'], vermogen_beluchting, vermogen_recirculatie))
    energieverbruik.append(bereken_energieverbruik(vermogen_beluchting, vermogen_recirculatie))  # kWh

effluent_df = influent_df.copy()
effluent_df['effluent_N_tot'] = effluent_n_tot
effluent_df['energieverbruik'] = energieverbruik
effluent_df['effluent_ok'] = np.where(effluent_df['effluent_N_tot'] <= 10, True, False)


totaal_energieverbruik = effluent_df['energieverbruik'].sum()
proportie_schoon_effluent = len(effluent_df[effluent_df['effluent_ok']]) / len(effluent_df)


print('Totaal energieverbruik: {:.0f}kWh'.format(totaal_energieverbruik))
print('Percentage van de tijd dat de effluentkwaliteit binnen de norm bleef: {:.1f}%'.format(100*proportie_schoon_effluent))